# Code to pre-process the .mp4 videos from the Drive&Act Dataset by extracting frames into .npy files with their labels. 

In [2]:
import os
import subprocess
import numpy as np
import cv2
import pandas as pd
import csv

## Define a one-hot-encoding helper function here:

In [3]:
# Might need to modify input "data_set"
def label_encoding(activity):
    if activity == 'sitting_still':
        activity = 0
    elif activity == 'standing_by_the_door':
        activity = 1
    elif activity == 'entering_car':
        activity = 2
    elif activity == 'closing_door_outside':
        activity = 3
    elif activity == 'closing_door_inside':
        activity = 4
    elif activity == 'opening_laptop':
        activity = 5
    elif activity == 'opening_door_inside':
        activity = 6
    elif activity == 'fetching_an_object':
        activity = 7
    elif activity == 'pressing_automation_button':
        activity = 8
    elif activity == 'fastening_seat_belt':
        activity = 9
    elif activity == 'using_multimedia_display':
        activity = 10
    elif activity == 'closing_laptop':
        activity = 11
    elif activity == 'placing_an_object':
        activity = 12
    elif activity == 'interacting_with_phone':
        activity = 13
    elif activity == 'drinking':
        activity = 14
    elif activity == 'opening_bottle':
        activity = 15
    elif activity == 'eating':
        activity = 16
    elif activity == 'preparing_food':
        activity = 17
    elif activity == 'looking_back_left_shoulder':
        activity = 18
    elif activity == 'reading_magazine':
        activity = 19
    elif activity == 'talking_on_phone':
        activity = 20
    elif activity == 'looking_or_moving_around (e.g. searching)':
        activity = 21
    elif activity == 'closing_bottle':
        activity = 22
    elif activity == 'putting_on_jacket':
        activity = 23
    elif activity == 'exiting_car':
        activity = 24
    elif activity == 'opening_door_outside':
        activity = 25
    elif activity == 'writing':
        activity = 26
    elif activity == 'working_on_laptop':
        activity = 27
    elif activity == 'putting_on_sunglasses':
        activity = 28
    elif activity == 'taking_off_sunglasses':
        activity = 29
    elif activity == 'opening_backpack':
        activity = 30
    elif activity == 'taking_laptop_from_backpack':
        activity = 31
    elif activity == 'reading_newspaper':
        activity = 32
    elif activity == 'taking_off_jacket':
        activity = 33
    elif activity == 'unfastening_seat_belt':
        activity = 34
    elif activity == 'putting_laptop_into_backpack':
        activity = 35
    else:
        print(activity)

    return int(activity)

## Test getting frames from single video for single row (activity slice) from the provided .csv file: 

In [4]:
path = "/Users/jasminesimmons/Grad_School/Spring_Qtr_2020/ECE285/Code_Final_Project/ECE285_Final_Dataset/activities_3s/kinect_ir/"
filename = "midlevel.chunks_90.split_0.train.csv"

In [5]:
!pwd

/Users/jasminesimmons/Grad_School/Spring_Qtr_2020/ECE285/Code_Final_Project/ECE285_Final_Dataset


In [6]:
os.path.exists(path + filename)

True

In [25]:
annotations_df = pd.read_csv(path + filename)
print(len(annotations_df))
annotations_df.head()

6642


,participant_id,file_id,annotation_id,frame_start,frame_end,activity,chunk_id
0,1,vp1/run1b_2018-05-29-14-02-47.kinect_ir,1,116,164,closing_door_outside,0
1,1,vp1/run1b_2018-05-29-14-02-47.kinect_ir,3,204,260,opening_door_outside,0
2,1,vp1/run1b_2018-05-29-14-02-47.kinect_ir,4,260,313,entering_car,0
3,1,vp1/run1b_2018-05-29-14-02-47.kinect_ir,5,313,348,closing_door_inside,0
4,1,vp1/run1b_2018-05-29-14-02-47.kinect_ir,6,348,439,fastening_seat_belt,0


In [10]:
video_path = "/Users/jasminesimmons/Grad_School/Spring_Qtr_2020/ECE285/Code_Final_Project/ECE285_Final_Dataset/kinect_ir/"
os.path.exists(video_path)

True

In [24]:
import cv2
import imageio

root_dir = "/Users/jasminesimmons/Grad_School/Spring_Qtr_2020/ECE285/Code_Final_Project/ECE285_Final_Dataset/kinect_ir"

for row in range(0, len(annotations_df)):
    #  Get video filename for each row
    video_filename = str(annotations_df["file_id"][row]) + ".mp4"        

    # Extract the frames using start_frame and end_frame
    start_frame = annotations_df["frame_start"][row]
    end_frame = annotations_df["frame_end"][row]

    activity = annotations_df["activity"][row]
    label = label_encoding(activity)
    
    video_path = root_dir + "/" + video_filename
    
    # Remove all files in prev. "proc" directory and recreate for this activity slice
    subprocess.call(["rm", "-rf", root_dir + "/proc"])
    subprocess.call(["mkdir", root_dir + "/proc"])
       
    ffmpeg = "/Users/jasminesimmons/Downloads/ffmpeg"
    subprocess.call([ffmpeg, "-i", video_path, "-r", "30", "-s", "224x224", "-aspect", "4:3", "proc/video_process.mp4"])

    #assuming 30 fps 
    fps = 30.0

    # each are specified in seconds
    start_time = float(start_frame) / float(fps)
    end_time = float(end_frame) / float(fps)
    duration = end_time - start_time
    
    # cut videos into activty chunk samples
    subprocess.call(["mkdir", root_dir + "/tmp"])
    subprocess.call([ffmpeg, "-i", root_dir + "/proc/video_process.mp4", "-ss", str((start_time)), "-to", str((end_time)), root_dir + "/tmp/video_process_cropped.mp4"])
    
    # sample the vids into 30 fps
    subprocess.call([ffmpeg, "-i", root_dir + "/tmp/video_process_cropped.mp4", "-r", "30", root_dir + "/tmp/thumb%04d.jpg"])
    
    start = 1
    end = int(fps * duration)

    # loop through all images, 30 fps for DURATION seconds
    for i in range(start, end):
        
        image_str = root_dir + "/tmp/thumb" + ('%04d' % i) + ".jpg"
        
        # Check to make sure that the file exists: 
        if os.path.exists(root_dir + "/tmp/thumb" + ('%04d' % i) + ".jpg"):
            gray_image = imageio.imread(image_str)

            # Convert to Greyscale
            # if (i == 1):
            #    prev_image = image

            # pretty sure our images are already gray, so not sure if we need these pre-processing steps:
            #gray_image_prev = cv2.cvtColor(prev_image, cv2.COLOR_BGR2GRAY)
            #gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Run optical flow: 
            '''
            dtvl1 = cv2.createOptFlow_DualTVL1()
            image_flowDTVL1 = dtvl1.calc(gray_image_prev, gray_image, None)    

            # Truncate Pixels between [-20, 20]
            image_flow_truncated = np.clip(image_flowDTVL1, -20, 20)

            # Normalize image into [-1, 1]
            norm_image_flow = cv2.normalize(image_flow_truncated, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

            # Crop 224x224 samples

            # reshape
            norm_image_flow_r = norm_image_flow.reshape(1, 224, 224, 2)
            '''

            norm_image_flow_r = gray_image

            # Add to Flow numpy
            if (i == 1):
                flow_video = norm_image_flow_r
            else:
                flow_video = np.concatenate((flow_video, norm_image_flow_r), axis=0)

            prev_image = image

            # print("Processed image " + str(i))

    #flow_video_r = flow_video.reshape(start, end, 224, 224, 2)

    # when done, copy .npy files for one 10s video clip, RGB and Flow
    subprocess.call(["mkdir", root_dir + "/video_flow/"])
    destination_flow = root_dir + "/video_flow/" + video_filename[4:] + "_" + str(start_frame) + str(end_frame) + ".npy"
    np.save(destination_flow, flow_video)

    with open(root_dir + '/video_flow/' + filename[:-3] + 'txt', "w") as txt_file: 
        txt_file.write(video_filename[4:] + "_" + str(start_frame) + str(end_frame) + ".npy" + " " + str(label) + "\n")

    if row == 1 or row == 2: 
        print("Processed video " + video_filename + " " + str(start_frame) + " " + str(end_frame))

print("Done processing!")

Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.kinect_ir.mp4 204 260
Processed video vp1/run1b_2018-05-29-14-02-47.ki

KeyboardInterrupt: 

## Create .npy from a single clip/activity slice: 

In [105]:
# TO DO: 
# 1. Add label (done)
# 2. Loop through all activity slices in .csv file
# 3. Optical FLOW and RGB Flow code? 

In [126]:
# Open a text file and record the .npy with the activity one-hot-encoding: 
# Place the file in: kinect_ir/video_flow/training.txt
txt_path = 'Users/jasminesimmons/Grad_School/Spring_Qtr_2020/ECE285/Code_Final_Project/ECE285_Final_Dataset/kinect_ir/video_flow/' + filename[:-3] + 'txt'
print(txt_path)

Users/jasminesimmons/Grad_School/Spring_Qtr_2020/ECE285/Code_Final_Project/ECE285_Final_Dataset/kinect_irvideo_flow/midlevel.chunks_90.split_0.train.txt


In [26]:
root_dir = "/Users/jasminesimmons/Grad_School/Spring_Qtr_2020/ECE285/Code_Final_Project/ECE285_Final_Dataset/kinect_ir"
os.path.exists(root_dir + '/video_flow/')

True

In [146]:

print(root_dir + '/video_flow/' + filename[:-3] + 'txt')
with open(root_dir + '/video_flow/' + filename[:-3] + 'txt', "w") as txt_file: 
    txt_file.write(video_filename[4:] + "_" + str(start_frame) + str(end_frame) + ".npy" + " " + str(label) + "\n")


/Users/jasminesimmons/Grad_School/Spring_Qtr_2020/ECE285/Code_Final_Project/ECE285_Final_Dataset/kinect_ir/video_flow/midlevel.chunks_90.split_0.train.txt


# //TO DO: (not implemented yet)
## Now do same as above, but loop through every row in the .csv file: 